In [1]:
%load_ext autoreload
%autoreload 2

In [22]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import KNNImputer

In [24]:
df = pd.read_csv('Data/pf_suvs_i302_1s2025.csv')
df = df.drop(df.columns[0], axis=1)
mask = df['Moneda'] == '$'
df.loc[mask, 'Precio'] = df.loc[mask, 'Precio'] / 1185
df.drop(columns=['Moneda'], inplace=True)
df.drop(columns = ['Tipo de carrocería'],inplace=True)
idx = df[df['Año'] == 436694.0].index
df.drop(idx, inplace=True)

df['Puertas'] = df['Puertas'].replace(60252.0, np.nan)
enc = OrdinalEncoder()
df['Modelo_code'] = enc.fit_transform(df[['Modelo']])
imputer = KNNImputer(n_neighbors=5)
cols = ['Modelo_code', 'Año', 'Puertas']
df[cols] = imputer.fit_transform(df[cols])
df['Puertas'] = np.round(df['Puertas']).astype(int)
df.drop('Modelo_code', axis=1, inplace=True)
print(df['Puertas'].value_counts(dropna=False))

Puertas
5    17734
4      399
3      105
2       10
6        3
7        2
Name: count, dtype: int64


In [ ]:
print(df.columns)


Index(['Marca', 'Modelo', 'Año', 'Versión', 'Color', 'Tipo de combustible',
       'Puertas', 'Transmisión', 'Motor', 'Con cámara de retroceso',
       'Kilómetros', 'Título', 'Precio', 'Descripción', 'Tipo de vendedor'],
      dtype='object')
Empty DataFrame
Columns: [Modelo, Puertas, Año]
Index: []
